In [ ]:
# 1. Chargement des données et des embeddings
# Cette première cellule va charger les données des livres et les embeddings,
# puis afficher quelques exemples pour vérifier que tout est correctement chargé.




import torch
import pandas as pd
from ScriptCompétence8 import charger_donnees_et_embeddings, calculate_or_load_cosine_similarity

# Charger les données des livres et les embeddings
data, embeddings = charger_donnees_et_embeddings()

# Vérification des dimensions des embeddings
print("Dimensions des embeddings:", embeddings.shape)
print("Nombre de livres dans data:", data.shape[0])

# Afficher quelques lignes pour vérifier les données
data.head()


c:\Users\pmgue\Downloads\ProjetChefDoeuvre\RecommendationsLectures\env\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Chargement des embeddings à partir du fichier


c:\Users\pmgue\Downloads\ProjetChefDoeuvre\RecommendationsLectures\RecommendationsLecturesAPI\ScriptCompétence8.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql_query("""


Dimensions des embeddings: torch.Size([6865, 384])
Nombre de livres dans data: 6481


c:\Users\pmgue\Downloads\ProjetChefDoeuvre\RecommendationsLectures\RecommendationsLecturesAPI\ScriptCompétence8.py:42: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  embeddin

,title,description,published_year,average_rating,thumbnail,authors,categories
0,Gilead,A NOVEL THAT READERS and critics have been eag...,2004,3.85,http://books.google.com/books/content?id=KQZCP...,[Marilynne Robinson],[Fiction]
1,Spider's Web,A new 'Christie for Christmas' -- a full-lengt...,2000,3.83,http://books.google.com/books/content?id=gA5GP...,"[Agatha Christie, Charles Osborne]",[Detective and mystery stories]
2,The One Tree,Volume Two of Stephen Donaldson's acclaimed se...,1982,3.97,http://books.google.com/books/content?id=OmQaw...,[Stephen R. Donaldson],[American fiction]
3,Rage of angels,"A memorable, mesmerizing heroine Jennifer -- b...",1993,3.93,http://books.google.com/books/content?id=FKo2T...,[Sidney Sheldon],[Fiction]
4,The Four Loves,Lewis' work on the nature of love divides love...,2002,4.15,http://books.google.com/books/content?id=XhQ5X...,[Clive Staples Lewis],[Christian life]


In [ ]:
# On a changé de csv donc probablement cela le soucis 
# Bon au moins on a le meme nombre de livres dans l'ancien csv et le nouveau avec les thumbnails donc le probleme est ailleurs

In [ ]:
import psycopg2
import pandas as pd

# Fonction pour se connecter à PostgreSQL
def get_db_connection():
    return psycopg2.connect(
        host="localhost",
        database="RecommendationsLectures",
        user="postgres",
        password="Lrk389229!"
    )

# Connexion à PostgreSQL et récupération des données sans `array_agg`
conn = get_db_connection()
query = """
    SELECT l.id_livre, l.title, l.description, l.published_year, l.average_rating, l.thumbnail,
           a.author_name AS author,
           c.category_name AS category
    FROM Livre l
    LEFT JOIN Livre_Auteur la ON l.id_livre = la.id_livre
    LEFT JOIN Auteur a ON la.id_auteur = a.id_auteur
    LEFT JOIN Livre_Categorie lc ON l.id_livre = lc.id_livre
    LEFT JOIN Categorie c ON lc.id_categorie = c.id_categorie
"""
data_expanded = pd.read_sql_query(query, conn)
conn.close()

# Vérifions le nombre de lignes retournées
print("Nombre de lignes dans data_expanded:", data_expanded.shape[0])

# Afficher un échantillon des données pour vérifier la structure
data_expanded.head()


In [ ]:
# 2. Calcul et chargement de la similarité cosinus
# Cette cellule va calculer (ou charger) les similarités cosinus entre tous les livres 
# et afficher la distribution des scores de similarité pour valider leur répartition.



import numpy as np
from ScriptCompétence8 import calculate_or_load_cosine_similarity

# Calculer ou charger les similarités cosinus
cosine_sim_embeddings = calculate_or_load_cosine_similarity(embeddings)

# Distribution des scores de similarité cosinus
similarity_scores = cosine_sim_embeddings.numpy().flatten()
print("Distribution des scores de similarité cosinus:", np.histogram(similarity_scores, bins=10))


In [ ]:

# 3. Fonction pour afficher les recommandations pour un livre donné
# La cellule ci-dessous utilise la fonction recommander_livres_sans_categorie pour générer des recommandations pour un titre donné,
# puis affiche les résultats et leurs scores de similarité.



from ScriptCompétence8 import recommander_livres_sans_categorie

# Fonction pour afficher les recommandations avec scores pour un livre donné
def afficher_recommandations(titre_livre, data, cosine_sim_embeddings, user_favorite_titles=[]):
    recommendations = recommander_livres_sans_categorie(titre_livre, data, cosine_sim_embeddings, user_favorite_titles)
    
    if not recommendations:
        print(f"Aucune recommandation trouvée pour le livre: {titre_livre}")
        return
    
    print(f"Recommandations pour '{titre_livre}':")
    for rec in recommendations:
        print(f"- Titre: {rec['title']}")
        print(f"  Score de similarité: {rec['score']:.4f}")
        print(f"  Auteurs: {rec['authors']}")
        print(f"  Année de publication: {rec['published_year']}")
        print(f"  Note moyenne: {rec['average_rating']}")
        print(f"  Catégories: {rec['categories']}\n")

# Test de la fonction avec un titre de livre spécifique
afficher_recommandations("The Lord of the Rings", data, cosine_sim_embeddings)


In [ ]:
# 4. Vérification sur plusieurs livres
# Pour vérifier les recommandations pour plusieurs livres, 
# vous pouvez exécuter cette cellule en changeant les titres de livres que vous souhaitez tester.




# Liste de livres pour lesquels on souhaite vérifier les recommandations
titres_de_test = ["The Lord of the Rings", "Pride and Prejudice", "Harry Potter", "Moby Dick", "The Great Gatsby"]

# Boucle pour afficher les recommandations pour chaque livre de la liste
for titre in titres_de_test:
    afficher_recommandations(titre, data, cosine_sim_embeddings)
    print("=" * 50)  # Séparateur entre chaque test


In [ ]:
# 5. Analyse des scores et des distributions
# Si vous voulez approfondir l'analyse des scores de similarité pour voir s'ils sont concentrés dans une certaine plage, 
# vous pouvez ajouter cette cellule qui montre les statistiques globales.




# Calcul des statistiques sur les scores de similarité
similarity_scores_flat = cosine_sim_embeddings.numpy().flatten()
print("Moyenne des scores de similarité:", np.mean(similarity_scores_flat))
print("Écart-type des scores de similarité:", np.std(similarity_scores_flat))
print("Score de similarité minimum:", np.min(similarity_scores_flat))
print("Score de similarité maximum:", np.max(similarity_scores_flat))

# Affichage d'un histogramme pour visualiser la répartition des scores
import matplotlib.pyplot as plt

plt.hist(similarity_scores_flat, bins=30, color='skyblue', edgecolor='black')
plt.title("Répartition des scores de similarité cosinus")
plt.xlabel("Score de similarité")
plt.ylabel("Fréquence")
plt.show()


In [ ]:
# 6. Normalisation optionnelle des embeddings
# Enfin, si les scores de similarité sont globalement trop élevés, 
# vous pouvez essayer de normaliser les embeddings, recalculer les similarités et comparer les résultats.



# Normalisation des embeddings
normalized_embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)

# Recalcul des similarités cosinus après normalisation
cosine_sim_embeddings_normalized = calculate_cosine_similarity_in_batches(normalized_embeddings)

# Test de recommandations avec les similarités normalisées
afficher_recommandations("The Lord of the Rings", data, cosine_sim_embeddings_normalized)
